<a href="https://colab.research.google.com/github/MatthieuHanania/PlasticDetection/blob/main/Dowloading_a_model_whithout_retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [2]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4305, done.
remote: Counting objects: 100% (4305/4305), done.
remote: Compressing objects: 100% (3290/3290), done.
remote: Total 4305 (delta 1210), reused 2184 (delta 942), pack-reused 0 (from 0)
Receiving objects: 100% (4305/4305), 53.16 MiB | 20.59 MiB/s, done.
Resolving deltas: 100% (1210/1210), done.
Updating files: 100% (3875/3875), done.


In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2024.8.2 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.


In [4]:
import io
import os
import cv2
import scipy.misc
import numpy as np
import six
import time

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

In [5]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Load the COCO Label Map
category_index = {
    1: {'id': 1, 'name': 'Plastic Bottle'},

}

In [6]:
# Download a model
# model zoo : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
!wget  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!tar -xf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8/ models/mymodel/

--2024-10-30 16:34:32--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.180.207, 142.251.16.207, 172.253.62.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.180.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M   145MB/s    in 0.3s    

2024-10-30 16:34:33 (145 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]



In [7]:
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('models/mymodel/saved_model/')

In [12]:
for image in images:
  print(os.path.basename(image),
        os.path.splitext(image))

IMG20200218113201.jpg ('test_images/IMG20200218113201', '.jpg')
IMG20200218112639.jpg ('test_images/IMG20200218112639', '.jpg')
IMG20200218113104.jpg ('test_images/IMG20200218113104', '.jpg')
IMG20200218112928.jpg ('test_images/IMG20200218112928', '.jpg')
IMG20200218112536.jpg ('test_images/IMG20200218112536', '.jpg')
IMG20200218112943.jpg ('test_images/IMG20200218112943', '.jpg')


In [13]:
image_path
os.path.basename(images[0])

image_fn = "result"+os.path.basename(image_path)
base_fn, ext = os.path.splitext(image_fn)
txt_result_fn = base_fn +'.txt'
txt_savepath = os.path.join(savepath, txt_result_fn)

NameError: name 'image_path' is not defined

In [ ]:
def export_img(image_path,image_np_with_detections, score):
    result_folder = "results/"
    image_name = "result_"+score+"_"+os.path.basename(image_path)
    savepath = os.path.join(result_folder, image_name)

    cv2.imwrite(savepath, image_np_with_detections)


In [17]:
import glob

test_images_folder = 'test_images'

images = glob.glob(test_images_folder+"/*.jpg")

for image_to_test_path in images:
  image_np = load_image_into_numpy_array(image_to_test_path)
  input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  plt.rcParams['figure.figsize'] = [42, 21]

  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)

  export_img(image_to_test_path,image_np_with_detections, detections['detection_scores'][0][0])

['test_images/IMG20200218113300.jpg',
 'test_images/IMG20200218113255.jpg',
 'test_images/IMG20200218113201.jpg',
 'test_images/IMG20200218112639.jpg',
 'test_images/IMG20200218113104.jpg',
 'test_images/IMG20200218112928.jpg',
 'test_images/IMG20200218112536.jpg',
 'test_images/IMG20200218113211.jpg',
 'test_images/IMG20200218112943.jpg']

In [ ]:


elapsed = []
for i in range(2):
  image_path = os.path.join(image_dir, 'image' + str(i + 1) + '.jpg')


  start_time = time.time()

  end_time = time.time()
  elapsed.append(end_time - start_time)
  a = str(i)
  os.chdir(directory)
  plt.rcParams['figure.figsize'] = [42, 21]
  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)
  #print(detections['detection_scores'][0][0].numpy(), 'llllll')
  #print(detections['detection_boxes'][0].numpy(), 'oooooooo')
  #print(detections['detection_classes'][0].numpy(), 'kkkkkk')
  plt.subplot(2, 1, i+1)
  plt.imshow(image_np_with_detections)
  export_img(image_np_with_detections, detections['detection_scores'][0][0])

mean_elapsed = sum(elapsed) / float(len(elapsed))
print('Elapsed time: ' + str(mean_elapsed) + ' second per image')